# Main ML code below

In [1]:
import numpy as np

from IPython.display import Image
from IPython.display import Markdown

import matplotlib
import matplotlib.pyplot as plt

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import torch.optim as optim

from torch.autograd import Variable

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

# material from notebook at: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../../../stanford_dogs_new/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
        
# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))
# 
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)
# 
# imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [3]:
regnet_y_3_2gf = models.regnet_y_3_2gf(pretrained=True)

for param in regnet_y_3_2gf.parameters():
    param.requires_grad = False

print(regnet_y_3_2gf)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            (1): B

In [4]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

regnet_y_3_2gf.fc = nn.Linear(in_features=1512, out_features=120, bias=True)

regnet_y_3_2gf = regnet_y_3_2gf.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_regnet_y_3_2gf = optim.SGD(regnet_y_3_2gf.parameters(), lr=0.001, momentum=0.9)
# optimizer_vgg19 = optim.Adamax(vgg19.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_regnet_y_3_2gf, step_size=7, gamma=0.1)

In [5]:
regnet_y_3_2gf = train_model(regnet_y_3_2gf, criterion, optimizer_regnet_y_3_2gf, exp_lr_scheduler, num_epochs=30)

# %


Epoch 0/29
----------
train Loss: 3.4391 Acc: 0.2283
val Loss: 1.3274 Acc: 0.6959

Epoch 1/29
----------
train Loss: 2.3388 Acc: 0.4242
val Loss: 0.7946 Acc: 0.7953

Epoch 2/29
----------
train Loss: 2.0591 Acc: 0.4814
val Loss: 0.6400 Acc: 0.8143

Epoch 3/29
----------
train Loss: 1.9432 Acc: 0.5085
val Loss: 0.6123 Acc: 0.8280

Epoch 4/29
----------
train Loss: 1.8533 Acc: 0.5200
val Loss: 0.5574 Acc: 0.8324

Epoch 5/29
----------
train Loss: 1.7824 Acc: 0.5382
val Loss: 0.5548 Acc: 0.8285

Epoch 6/29
----------
train Loss: 1.7591 Acc: 0.5434
val Loss: 0.5285 Acc: 0.8397

Epoch 7/29
----------
train Loss: 1.5836 Acc: 0.6001
val Loss: 0.4921 Acc: 0.8523

Epoch 8/29
----------
train Loss: 1.5616 Acc: 0.6068
val Loss: 0.4677 Acc: 0.8514

Epoch 9/29
----------
train Loss: 1.5519 Acc: 0.6051
val Loss: 0.4776 Acc: 0.8548

Epoch 10/29
----------
train Loss: 1.5516 Acc: 0.6083
val Loss: 0.4762 Acc: 0.8499

Epoch 11/29
----------
train Loss: 1.5325 Acc: 0.6076
val Loss: 0.4768 Acc: 0.8558

Ep

In [6]:
torch.save(regnet_y_3_2gf.state_dict(), 'regnet_y_3_2gf_based_model2.pt')

## Load the Model and test it

In [7]:
regnet_y_3_2gf_loaded = models.regnet_y_3_2gf(pretrained=True)

for param in regnet_y_3_2gf_loaded.parameters():
    param.requires_grad = False

regnet_y_3_2gf_loaded.fc = nn.Linear(in_features=1512, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
regnet_y_3_2gf_loaded = regnet_y_3_2gf_loaded.to(device)



regnet_y_3_2gf_loaded.load_state_dict(torch.load('regnet_y_3_2gf_based_model2.pt'), strict=False)
regnet_y_3_2gf_loaded.eval()

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            (1): B